In [1]:
import pandas as pd
import datetime
import numpy as np
import gc

In [2]:
SOTF_df=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Compare_Q4_Before_After_Groups_JT_20190327/Q4 SOTF Master Zip Doc.xlsx",
                      dtype=str,sheet_name="Store ID List",header=None)
SOTF_df.columns=['location_id']
SOTF_df['store_type']="SOTF"

SOTF_df=SOTF_df.append(pd.DataFrame({"location_id":"6990","store_type":"Ecommerce"},index=[0]))
SOTF_df=SOTF_df.reset_index()
del SOTF_df['index']

In [3]:
product_taxonomy=pd.read_table("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20190501-135142-836.txt",dtype=str,sep="|")
product_taxonomy=product_taxonomy[['division_id','class_code_id','subclass_id']].drop_duplicates()
division_table=product_taxonomy.copy()

divison_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Division Names.txt",dtype=str,sep="|")


In [4]:
df_2018Q4_sales_all=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Compare_Q4_Before_After_Groups_JT_20190327/Q4_rewards_sales_trans.csv",dtype=str)
df_2018Q4_sales_all['item_transaction_units']=df_2018Q4_sales_all['item_transaction_units'].astype(int)
df_2018Q4_sales_all['item_transaction_amt']=df_2018Q4_sales_all['item_transaction_amt'].astype(float)
df_2018Q4_sales_all=pd.merge(df_2018Q4_sales_all,product_taxonomy,on=["class_code_id",'subclass_id'],how="left")
df_2018Q4_sales_all.head(2)

,location_id,transaction_dt,transaction_id,customer_id_hashed,class_code_id,subclass_id,item_transaction_units,item_transaction_amt,division_id
0,4621,2018-11-10,3371,34e22ad0c4e216b4983bc2978770b3dbd7301213f8ebfd...,12006,1,2,3.0,2
1,4642,2018-11-10,3249,049f7c29a72fd252a880b32912fa49adc5dc55f3047193...,36007,1,1,32.0,3


In [5]:
df_2018Q4_sales_all=pd.merge(df_2018Q4_sales_all,SOTF_df,on="location_id",how="left")
df_2018Q4_sales_all['store_type']=df_2018Q4_sales_all['store_type'].fillna("Legacy")
print(df_2018Q4_sales_all.shape)
import gc
gc.collect()

(135159718, 10)


3748

In [6]:
id_groups=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Compare_Q4_Before_After_Groups_JT_20190327/BL_Q4_rewards_migration_rewards_data_JL_2019-03-30.csv",
                     dtype=str,usecols=['customer_id_hashed','Group_before_Q4','Q4_Shopping_Group','NewRewards'])

id_groups=id_groups[id_groups['Q4_Shopping_Group']=="Q4_Shopped"]
del id_groups['Q4_Shopping_Group']
gc.collect()


df_2018Q4_sales_all=pd.merge(df_2018Q4_sales_all,id_groups,on="customer_id_hashed",how="left")
print(df_2018Q4_sales_all.shape)
del id_groups
gc.collect()

(135159718, 12)


21

In [7]:
df_2018Q4_sales_all.head(2)

,location_id,transaction_dt,transaction_id,customer_id_hashed,class_code_id,subclass_id,item_transaction_units,item_transaction_amt,division_id,store_type,Group_before_Q4,NewRewards
0,4621,2018-11-10,3371,34e22ad0c4e216b4983bc2978770b3dbd7301213f8ebfd...,12006,1,2,3.0,2,Legacy,Active,Old_Rewards_Members
1,4642,2018-11-10,3249,049f7c29a72fd252a880b32912fa49adc5dc55f3047193...,36007,1,1,32.0,3,Legacy,Active,Old_Rewards_Members


In [8]:
df_2018Q4_sales_all.columns.tolist()

['location_id',
 'transaction_dt',
 'transaction_id',
 'customer_id_hashed',
 'class_code_id',
 'subclass_id',
 'item_transaction_units',
 'item_transaction_amt',
 'division_id',
 'store_type',
 'Group_before_Q4',
 'NewRewards']

In [9]:
# print(df_2018Q4_sales_all['Group_after_Q4'].unique())
# del df_2018Q4_sales_all['Group_after_Q4']
del df_2018Q4_sales_all['class_code_id']
del df_2018Q4_sales_all['subclass_id']
gc.collect()

272

In [10]:
df_2018Q4_sales_all.head(2)

,location_id,transaction_dt,transaction_id,customer_id_hashed,item_transaction_units,item_transaction_amt,division_id,store_type,Group_before_Q4,NewRewards
0,4621,2018-11-10,3371,34e22ad0c4e216b4983bc2978770b3dbd7301213f8ebfd...,2,3.0,2,Legacy,Active,Old_Rewards_Members
1,4642,2018-11-10,3249,049f7c29a72fd252a880b32912fa49adc5dc55f3047193...,1,32.0,3,Legacy,Active,Old_Rewards_Members


In [11]:
gc.collect()

221

In [12]:
print(len(df_2018Q4_sales_all['customer_id_hashed'].unique()))
gc.collect()

9777953


0

In [13]:
# 1. by by division, by id group

# Sales & Units
df_2018Q4_sales_div_agg=df_2018Q4_sales_all.groupby(['Group_before_Q4','NewRewards','division_id'])['item_transaction_amt','item_transaction_units'].sum().reset_index().rename(columns={"item_transaction_amt":"sales",'item_transaction_units':"units"})
# Trans
df_2018Q4_trans_div_agg=df_2018Q4_sales_all[['location_id','transaction_dt','transaction_id','customer_id_hashed','division_id','Group_before_Q4','NewRewards']].drop_duplicates()
df_2018Q4_trans_div_agg=df_2018Q4_trans_div_agg.groupby(['Group_before_Q4','NewRewards','division_id'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"transactions"})
# Ids
df_2018Q4_ids_div_agg=df_2018Q4_sales_all[['division_id','Group_before_Q4','NewRewards','customer_id_hashed']].drop_duplicates()
df_2018Q4_ids_div_agg=df_2018Q4_ids_div_agg.groupby(['Group_before_Q4','NewRewards','division_id'])['customer_id_hashed'].count().to_frame().reset_index().rename(columns={"customer_id_hashed":"id_count"})


output_division_with_id=pd.merge(df_2018Q4_sales_div_agg,df_2018Q4_trans_div_agg,on=['Group_before_Q4','NewRewards','division_id'],how="outer")
output_division_with_id=pd.merge(output_division_with_id,df_2018Q4_ids_div_agg,on=['Group_before_Q4','NewRewards','division_id'],how="outer")

print(output_division_with_id.shape)
output_division_with_id.head(4)


(40, 7)


,Group_before_Q4,NewRewards,division_id,sales,units,transactions,id_count
0,Active,Old_Rewards_Members,1,1.172652e+08,53981704,11099948,5248593
1,Active,Old_Rewards_Members,10,1.760110e+05,733207,545998,337949
2,Active,Old_Rewards_Members,2,1.148151e+08,33546777,9323470,4781047
3,Active,Old_Rewards_Members,3,1.160921e+08,20595435,5764876,3461013


In [14]:
# 2. by by store types, by id group
# Sales & Units
df_2018Q4_sales_store_agg=df_2018Q4_sales_all.groupby(['Group_before_Q4','NewRewards','store_type'])['item_transaction_amt','item_transaction_units'].sum().reset_index().rename(columns={"item_transaction_amt":"sales",'item_transaction_units':"units"})
# Trans
df_2018Q4_trans_store_agg=df_2018Q4_sales_all[['location_id','transaction_dt','transaction_id','customer_id_hashed','store_type','Group_before_Q4','NewRewards']].drop_duplicates()
df_2018Q4_trans_store_agg=df_2018Q4_trans_store_agg.groupby(['Group_before_Q4','NewRewards','store_type'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"transactions"})
# Ids
df_2018Q4_ids_store_agg=df_2018Q4_sales_all[['customer_id_hashed','store_type','Group_before_Q4','NewRewards']].drop_duplicates()
df_2018Q4_ids_store_agg=df_2018Q4_ids_store_agg.groupby(['Group_before_Q4','NewRewards','store_type'])['customer_id_hashed'].count().to_frame().reset_index().rename(columns={"customer_id_hashed":"id_count"})


output_store_with_id=pd.merge(df_2018Q4_sales_store_agg,df_2018Q4_trans_store_agg,on=['Group_before_Q4','NewRewards','store_type'],how="outer")
output_store_with_id=pd.merge(output_store_with_id,df_2018Q4_ids_store_agg,on=['Group_before_Q4','NewRewards','store_type'],how="outer")

print(output_store_with_id.shape)
output_store_with_id.head(4)

(15, 7)


,Group_before_Q4,NewRewards,store_type,sales,units,transactions,id_count
0,Active,Old_Rewards_Members,Ecommerce,3.666583e+06,163921,47563,33156
1,Active,Old_Rewards_Members,Legacy,5.879497e+08,120484391,16360803,6214751
2,Active,Old_Rewards_Members,SOTF,1.039564e+08,20056035,2697974,1105207
3,Active,Q4_New_Sign_Ups,Ecommerce,2.999000e+01,1,1,1


In [15]:
# 3. by store type, by division, by id group
df_2018Q4_sales_div_store_agg=df_2018Q4_sales_all.groupby(['Group_before_Q4','NewRewards','division_id','store_type'])['item_transaction_amt','item_transaction_units'].sum().reset_index().rename(columns={"item_transaction_amt":"sales",'item_transaction_units':"units"})
# Trans
df_2018Q4_trans_div_store_agg=df_2018Q4_sales_all[['location_id','transaction_dt','transaction_id','customer_id_hashed','division_id','store_type','Group_before_Q4','NewRewards']].drop_duplicates()
df_2018Q4_trans_div_store_agg=df_2018Q4_trans_div_store_agg.groupby(['Group_before_Q4','NewRewards','division_id','store_type'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"transactions"})
# Ids
df_2018Q4_ids_div_store_agg=df_2018Q4_sales_all[['customer_id_hashed','division_id','store_type','Group_before_Q4','NewRewards']].drop_duplicates()
df_2018Q4_ids_div_store_agg=df_2018Q4_ids_div_store_agg.groupby(['Group_before_Q4','NewRewards','division_id','store_type'])['customer_id_hashed'].count().to_frame().reset_index().rename(columns={"customer_id_hashed":"id_count"})


output_div_store_with_id=pd.merge(df_2018Q4_sales_div_store_agg,df_2018Q4_trans_div_store_agg,on=['Group_before_Q4','NewRewards','division_id','store_type'],how="outer")
output_div_store_with_id=pd.merge(output_div_store_with_id,df_2018Q4_ids_div_store_agg,on=['Group_before_Q4','NewRewards','division_id','store_type'],how="outer")

print(output_div_store_with_id.shape)
output_div_store_with_id.head(4)

(108, 8)


,Group_before_Q4,NewRewards,division_id,store_type,sales,units,transactions,id_count
0,Active,Old_Rewards_Members,1,Ecommerce,1.607292e+05,6694,2548,2289
1,Active,Old_Rewards_Members,1,Legacy,1.008324e+08,46533014,9569293,4538361
2,Active,Old_Rewards_Members,1,SOTF,1.627212e+07,7441996,1528107,784631
3,Active,Old_Rewards_Members,10,Legacy,1.577825e+05,668565,498155,307532


In [25]:
output_division_with_id=pd.merge(output_division_with_id,divison_name,on="division_id",how="left")
output_div_store_with_id=pd.merge(output_div_store_with_id,divison_name,on="division_id",how="left")




In [26]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Compare_Q4_Before_After_Groups_JT_20190327/BL_Q4_rewards_summary_div_store_type_groups_data_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
output_division_with_id.to_excel(writer,"output_division_with_id",index=False)
output_store_with_id.to_excel(writer,"output_store_with_id",index=False)
output_div_store_with_id.to_excel(writer,"output_div_store_with_id",index=False)
writer.save()
